# 1. Install Dependencies and Setup

In [ ]:
import tensorflow as tf
import os
from matplotlib import pyplot as plt
from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
from sklearn.model_selection import GridSearchCV

In [ ]:
# Avoid OOM errors by setting GPU Memory Consumption Growth
gpus = tf.config.experimental.list_physical_devices('GPU')
for gpu in gpus: 
    tf.config.experimental.set_memory_growth(gpu, True)

In [ ]:
tf.config.list_physical_devices('CPU')

# 2. Remove dodgy images

In [ ]:
import cv2
import imghdr

In [ ]:
data_dir = r"C:\Users\HP\Desktop\Online Courses\IBM Machine Learning Engineer\4. Deep Learning and Reinforcement Learning\Week 9\projects\nemophea\data"

In [ ]:
os.listdir(data_dir)

In [ ]:
os.listdir(os.path.join(data_dir, 'Normal'))

In [ ]:
image_exts = ['jpeg','jpg', 'bmp', 'png']

In [ ]:
img=cv2.imread(os.path.join(data_dir, 'Normal', 'IM-0001-0001.jpeg'))

In [ ]:
plt.imshow(img)
plt.show

In [ ]:
rgb_image = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
plt.imshow(rgb_image)
plt.show()

In [ ]:
for image_class in os.listdir(data_dir): 
    for image in os.listdir(os.path.join(data_dir, image_class)):
        image_path = os.path.join(data_dir, image_class, image)
        try: 
            img = cv2.imread(image_path)
            tip = imghdr.what(image_path)
            if tip not in image_exts: 
                print('Image not in ext list {}'.format(image_path))
                os.remove(image_path)
        except Exception as e: 
            print('Issue with image {}'.format(image_path))
            # os.remove(image_path)

# 3. Load Data

In [ ]:
import numpy as np
from matplotlib import pyplot as plt

In [ ]:
data = tf.keras.utils.image_dataset_from_directory(data_dir)

In [ ]:
data_iterator = data.as_numpy_iterator()

In [ ]:
batch = data_iterator.next()

In [ ]:
batch[0].max()

In [ ]:
# Obtain class names from the dataset
class_names = data.class_names

# Print the class names
print("Class names:", class_names)

len(class_names)

labels=list(range(len(class_names)))
print(labels)

for label in labels:
    class_name = class_names[label]
    print(f"Label {label} corresponds to class: {class_name}")

In [ ]:
fig, ax = plt.subplots(ncols=4, figsize=(20,20))
for idx, img in enumerate(batch[0][:4]):
    ax[idx].imshow(img.astype(int))
    ax[idx].title.set_text(batch[1][idx])

# 4. Scale Data

In [ ]:
data = data.map(lambda x,y: (x/255, y))

In [ ]:
scaled_iterator = data.as_numpy_iterator()

batch=scaled_iterator.next()

batch[0].min(), batch[0].max()

# 5. Split Data

In [ ]:
len(data)

In [ ]:
train_size = int(len(data)*.7)
val_size = int(len(data)*.2)
test_size = int(len(data)*.1)+1

In [ ]:
test_size+train_size+val_size

In [ ]:
train = data.take(train_size)
val = data.skip(train_size).take(val_size)
test = data.skip(train_size+val_size).take(test_size)

In [ ]:
# import tensorflow as tf
# import numpy as np

# # Load the dataset
# data = tf.keras.utils.image_dataset_from_directory(data_dir)

# # Initialize empty lists to store features and labels
# X_train = []
# y_train = []

# # Concatenate features and labels from the dataset into X_train and y_train
# for images, labels in data:
#     X_train.append(images.numpy())  # Append images to X_train
#     y_train.append(labels.numpy())  # Append labels to y_train

# # Convert lists to NumPy arrays
# X_train = np.concatenate(X_train, axis=0)
# y_train = np.concatenate(y_train, axis=0)

# # Display shapes of X_train and y_train
# print("X_train shape:", X_train.shape)
# print("y_train shape:", y_train.shape)


# 6. Build Deep Learning Model

In [ ]:
train

In [ ]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Dense, Flatten, Dropout

In [ ]:
# GRID SEARCH

# from tensorflow.keras.models import Sequential
# from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense
# from tensorflow.keras.wrappers.scikit_learn import KerasClassifier
# from sklearn.model_selection import GridSearchCV

# # Function to create CNN model
# def create_model(num_conv_layers=1, filters=16, kernel_size=(3, 3), pool_size=(2, 2), activation='relu'):
#     model = Sequential()
#     model.add(Conv2D(filters=filters, kernel_size=kernel_size, activation=activation, input_shape=(256, 256, 3)))
#     model.add(MaxPooling2D(pool_size=pool_size))
    
#     # Add additional convolutional layers based on num_conv_layers
#     for _ in range(num_conv_layers - 1):
#         model.add(Conv2D(filters=filters, kernel_size=kernel_size, activation=activation))
#         model.add(MaxPooling2D(pool_size=pool_size))
    
#     model.add(Flatten())
#     model.add(Dense(256, activation='relu'))
#     model.add(Dense(1, activation='sigmoid'))
    
#     model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])
#     return model

# # Create KerasClassifier wrapper for use in GridSearchCV
# model = KerasClassifier(build_fn=create_model, verbose=0)

# # Define hyperparameters grid including the number of convolutional layers
# param_grid = {
#     'num_conv_layers': [1, 2, 3],  # Number of convolutional layers to test
#     'filters': [16, 32],
#     'kernel_size': [(3, 3), (5, 5)],
#     'pool_size': [(2, 2), (3, 3)],
#     'activation': ['relu', 'sigmoid']
# }

# # Perform GridSearchCV
# grid = GridSearchCV(estimator=model, param_grid=param_grid, cv=3, verbose=1)
# grid_result = grid.fit(X_train, y_train)  # X_train, y_train: your training data

# # Summarize results
# print("Best: %f using %s" % (grid_result.best_score_, grid_result.best_params_))


In [ ]:
model = Sequential()
model.add(Conv2D(16, (3,3), 1, activation='relu', input_shape=(256,256,3)))
model.add(MaxPooling2D())
model.add(Conv2D(32, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Conv2D(16, (3,3), 1, activation='relu'))
model.add(MaxPooling2D())
model.add(Flatten())
model.add(Dense(256, activation='relu'))
model.add(Dense(1, activation='sigmoid'))
model.compile('adam', loss=tf.losses.BinaryCrossentropy(), metrics=['accuracy'])

In [ ]:
model.summary()

# 7. Train

In [ ]:
logdir='logs'

In [ ]:
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)

In [ ]:
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])

# 8. Plot Performance

In [ ]:
fig = plt.figure()
plt.plot(hist.history['loss'], color='teal', label='loss')
plt.plot(hist.history['val_loss'], color='orange', label='val_loss')
fig.suptitle('Loss', fontsize=20)
plt.legend(loc="upper left")
plt.show()

In [ ]:
fig = plt.figure()
plt.plot(hist.history['accuracy'], color='teal', label='accuracy')
plt.plot(hist.history['val_accuracy'], color='orange', label='val_accuracy')
fig.suptitle('Accuracy', fontsize=20)
plt.legend(loc="upper left")
plt.show()

# 9. Evaluate

In [ ]:
from tensorflow.keras.metrics import Precision, Recall, BinaryAccuracy

In [ ]:
pre = Precision()
re = Recall()
acc = BinaryAccuracy()

In [ ]:
for batch in test.as_numpy_iterator(): 
    X, y = batch
    yhat = model.predict(X)
    pre.update_state(y, yhat)
    re.update_state(y, yhat)
    acc.update_state(y, yhat)

In [ ]:
print(f'Precision: {pre.result().numpy()}, Recall: {re.result().numpy()}, Accuracy: {acc.result().numpy()}')

# 10. Test

In [ ]:
import cv2

In [ ]:
img = cv2.imread(r"C:\Users\HP\Desktop\Online Courses\IBM Machine Learning Engineer\4. Deep Learning and Reinforcement Learning\Week 9\project\TESTING\NORMAL.jpg")
plt.imshow(img)
plt.show()

In [ ]:
resize = tf.image.resize(img, (256,256))
plt.imshow(resize.numpy().astype(int))
plt.show()

In [ ]:
yhat = model.predict(np.expand_dims(resize/255, 0))

In [ ]:
yhat

In [ ]:
if yhat > 0.5: 
    print(f'Predicted Results : Kneumonia')
else:
    print(f'Predicted Results: Normal')

In [ ]:
def predict_kneumonia(image):
    img = cv2.imread(image)
    plt.imshow(img)
    plt.show()
    resize = tf.image.resize(img, (256,256))
    yhat = model.predict(np.expand_dims(resize/255, 0))
    if yhat > 0.5: 
        print(f'Predicted Results : Kneumonia')
    else:
        print(f'Predicted Results : Normal')

In [ ]:
predict_kneumonia(r"C:\Users\HP\Desktop\Online Courses\IBM Machine Learning Engineer\4. Deep Learning and Reinforcement Learning\Week 9\project\TESTING\PNEUMONIA.jpg")

# 11. Save the Model

In [ ]:
from tensorflow.keras.models import load_model

In [ ]:
model.save(os.path.join('models','imageclassifier.h5'))

In [ ]:
new_model = load_model('imageclassifier.h5')

In [ ]:
new_model.predict(np.expand_dims(resize/255, 0))

# ANN

In [ ]:
# Initialize the model
model = Sequential()

# Add Dense layers (fully connected layers)
model.add(Dense(64, activation='relu', input_shape=(256 * 256 * 3,)))  # Input layer
model.add(Dense(32, activation='relu'))  # Hidden layer
model.add(Dense(1, activation='sigmoid'))  # Output layer for binary classification

# Compile the model
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# Display model summary
model.summary()

In [ ]:
logdir='logs'
tensorboard_callback = tf.keras.callbacks.TensorBoard(log_dir=logdir)
hist = model.fit(train, epochs=20, validation_data=val, callbacks=[tensorboard_callback])